In [92]:
from spotpy.parameter import Uniform
import spotpy
import pandas as pd
import os
import hsamiplus
from spotpy.objectivefunctions import rmse

In [ ]:
# Dossier où se trouvent les fichiers
data_dir = os.path.join('..','..', 'data')

#Fichier paramètres
param_file = os.path.join(data_dir, 'parametres.txt')
df_param = pd.read_csv(param_file, header=0, sep='\s+')
df_param.columns = ["Nom", "min", "default", "max"]

In [94]:
# Transformation de paramètres en fonctions uniformes pour spotpy
params = {}
for p in df_param.index:
    params[df_param.loc[p]['Nom']] = Uniform(low=df_param.loc[p]['min'].item() , high=df_param.loc[p]['max'].item())
    # print(df_param.loc[p]['Nom'] + ' = ' + 'Uniform(low=' + str(df_param.loc[p]['min'].item()) + ', high=' + str(df_param.loc[p]['max'].item())+ ')')

In [95]:
# Projet de 5 ans
import json
from pathlib import Path
 
filename = '5_ans.json'
with Path.open(Path(data_dir) / filename) as file:
    projet = json.load(file)

In [96]:
# On sauve la sim pour avoir des faux obs
simulation, etats, deltas = hsamiplus.hsami2(projet)
pd.DataFrame(simulation)['Qtotal'].to_csv('5ans_sim_output.csv')

In [ ]:
# Dans le projet, on remplace les params par les fcts uniformes, pas sûr que ce soit nécéssaire...
projet['param'] = params

In [98]:
# Classe du modèle hsami+
class spot_setup(object):   
    # Vraiment pas propre, mais pour l'instant c'est ce qui est le plus simple, 
    # sinon je ne suis pas sûr comment faire vu que c'est avant le init
    efficacite_evapo_ete = Uniform(low=0.5, high=1.75)
    efficacite_evapo_hiver = Uniform(low=0.0, high=0.3)
    taux_fonte_jour = Uniform(low=0.1, high=0.5)
    taux_fonte_nuit = Uniform(low=0.05, high=0.3)
    temp_fonte_jour = Uniform(low=-4.0, high=4.0)
    temp_fonte_nuit = Uniform(low=-4.0, high=4.0)
    temp_ref_pluie = Uniform(low=-2.0, high=2.0)
    effet_redoux = Uniform(low=1.1, high=3.0)
    effet_gel = Uniform(low=1.0, high=10.0)
    effet_sol = Uniform(low=5.0, high=35.0)
    seuil_min = Uniform(low=1.0, high=6.0)
    sol_min = Uniform(low=0.05, high=10.0)
    sol_max = Uniform(low=10.0, high=50.0)
    nappe_max = Uniform(low=8.0, high=50.0)
    portion_ruissellement_surface = Uniform(low=0.25, high=0.55)
    portion_ruissellement_sol_max = Uniform(low=0.2, high=0.8)
    taux_vidange_sol_min = Uniform(low=0.01, high=0.04)
    taux_vidange_nappe = Uniform(low=0.008, high=0.025)
    taux_vidange_inter = Uniform(low=0.4, high=0.95)
    mode_hydrogramme_surface = Uniform(low=0.5, high=3.0)
    forme_hydrogramme_surface = Uniform(low=0.7, high=3.0)
    mode_hydrogramme_inter = Uniform(low=1.0, high=10.0)
    forme_hydrogramme_inter = Uniform(low=0.3, high=4.0)
    curve_number = Uniform(low=25.0, high=100.0)
    puissance_K_s_1 = Uniform(low=-3.0, high=4.0)
    potentiel_matriciel = Uniform(low=1.0, high=150.0)
    coeff_recession_nappe = Uniform(low=0.6, high=0.999)
    rendement_spécifique_nappe = Uniform(low=0.01, high=0.5)
    taux_de_fonte_1 = Uniform(low=0.1, high=0.6)
    taux_de_fonte_2 = Uniform(low=0.01, high=0.4)
    taux_de_fonte_3 = Uniform(low=0.01, high=0.4)
    temperature_fonte_1 = Uniform(low=-4.0, high=4.0)
    temperature_fonte_2 = Uniform(low=-2.0, high=0.0)
    temperature_fonte_3 = Uniform(low=-2.0, high=0.0)
    puissance_K_s_surf = Uniform(low=-4.0, high=-1.0)
    capacite_retenue = Uniform(low=0.02, high=0.13)
    indice_distr_taille_pores_1 = Uniform(low=4.0, high=12.0)
    indice_distr_taille_pores_2 = Uniform(low=4.0, high=12.0)
    puissance_K_s_2 = Uniform(low=-3.0, high=4.0)
    epaisseur_couche_1 = Uniform(low=5.0, high=100.0)
    epaisseur_couche_2 = Uniform(low=10.0, high=500.0)
    point_fletrissement = Uniform(low=0.05, high=0.25)
    capacite_champ_1 = Uniform(low=0.1, high=0.3)
    capacite_champ_2 = Uniform(low=0.1, high=0.3)
    porosite_couche_1 = Uniform(low=0.1, high=0.4)
    porosite_couche_2 = Uniform(low=0.1, high=0.4)
    coeff_stefan = Uniform(low=0.7, high=2.7)
    coeff_volume_max = Uniform(low=1.0, high=3.0)
    coeff_surface_normale = Uniform(low=0.1, high=0.4)
    puissance_cond_hydrau_sat = Uniform(low=-2.0, high=-1.0)
    def __init__(self, projet, obj_func=None):

            # On lit les obs, tant qu'à y être, ça pourrait être dans le projet
            self.projet = projet
            #Load Observation data from file
            obsfile = open(data_dir+os.sep+'5ans_sim_output.csv', 'r')
            headerline = obsfile.readline()[:-1]
            self.trueObs = []
            #Read model forcing in working storage (this is done only ones)
            if ';' in headerline:
                self.delimiter = ';'
            else:
                self.delimiter = ','
            self.header = headerline.split(self.delimiter)
            for line in obsfile:
                values =  line.strip().split(self.delimiter)
                self.trueObs.append(float(values[0]))

            obsfile.close()
            self.obj_func = obj_func

    def simulation(self, params):
    #Here the model is actualy started with a unique parameter combination that it gets from spotpy for each time the model is called
        projet = self.projet
        projet['param'] = params
        simulation, etats, deltas = hsamiplus.hsami2(projet)
        sim = simulation['Qtotal']
        #The first year of simulation data is ignored (warm-up) 
        return sim[366:]

    def evaluation(self):
        #The first year of simulation data is ignored (warm-up)
        return self.trueObs[366:]
    
    # Généraliser pour voir les différentes fcts de spotpy
    def objectivefunction(self,simulation,evaluation, params=None):
        #SPOTPY expects to get one or multiple values back, 
        #that define the performance of the model run
        if not self.obj_func:
            # This is used if not overwritten by user
            like = rmse(evaluation,simulation)
        else:
            #Way to ensure flexible spot setup class
            like = self.obj_func(evaluation,simulation)    
        return like

In [99]:
hs_spot = spot_setup(projet)

In [ ]:
sampler = spotpy.algorithms.dds(
    hs_spot, dbname='hsami_model_run', dbformat='ram', save_sim=False
)

# Launch the actual optimization. Multiple trials can be launched, where the entire process is repeated and
# the best overall value from all trials is returned.


##########################################
## Calibration                          ##
##########################################
sampler.sample(20, trials=1)

##########################################
## Post-calibration                     ##
##########################################
# Get all the values of each iteration
results = sampler.getdata()

bestindex, bestobjfun = spotpy.analyser.get_maxlikeindex(
    results
)  # Want to get the MAX (change for min for RMSE)
best_model_run = list(
    results[bestindex][0]
)  # Get the parameter set returning the best KGE
optimized_parameters = best_model_run[
    1:-1]